In [72]:
import re

import pandas as pd
import numpy as np

In [73]:
lovd_variants = pd.read_table("/Users/himanshujoshi/Desktop/LOVD variants/all_LOVD_variants.tsv.gz", sep="\t")
refseq_tx = pd.read_table("refseq_tx_chromosome_map.txt.gz", sep="\t")

# Prep Refseq data

In [74]:
np.any(refseq_tx.isna(), axis=1).value_counts()

False    35925
True        25
dtype: int64

In [75]:
refseq_tx.dropna(inplace=True)

In [76]:
np.any(refseq_tx.isna(), axis=1).value_counts()

False    35925
dtype: int64

In [77]:
refseq_tx.head()

,Chromosome/scaffold name,RefSeq mRNA ID
1,1,NM_152499
2,1,NM_173638
3,1,NM_001170755
4,1,NM_024097
5,1,NM_001242750


# Prep Variants

## Infer variant type (SNP or other) using regex

In [78]:
snp_pattern = re.compile("g.(\d+)([ATCG])>([ATCG])")

In [79]:
lovd_variants['Variant_Type'] = lovd_variants.DNA_change_genomic_hg19.apply(lambda x: 'SNP' if snp_pattern.match(x) is not None else 'Other')

In [80]:
lovd_variants.Variant_Type.value_counts()

SNP      260251
Other     61750
Name: Variant_Type, dtype: int64

## Obtain chromosome based on mRNA to Chr mapping (downloaded from GRCh37 Ensembl biomart)

In [81]:
lovd_variants['RefSeq mRNA ID'] = lovd_variants.Transcript.str.split('.', expand=True)[0]

In [82]:
lovd_variants = lovd_variants.merge(refseq_tx, how='left', on='RefSeq mRNA ID')

## Get chr/pos/ref/alt for SNP variants

In [89]:
snp_pos = lovd_variants.loc[lovd_variants['Variant_Type']=='SNP']['DNA_change_genomic_hg19'].apply(lambda x: pd.Series({'pos': snp_pattern.match(x)[1],
                                                                                                              'ref': snp_pattern.match(x)[2],
                                                                                                              'alt': snp_pattern.match(x)[3]}))

In [90]:
lovd_variants = lovd_variants.join(snp_pos)

## Create a hgvs_g column so that we can use it to obtain VCF coordinates for non-SNP variants (Keep in mind, some of these will be invalid entries)

In [94]:
lovd_variants.columns

Index(['DB_ID', 'Gene', 'Transcript', 'DNA_change_cDNA',
       'DNA_change_genomic_hg19', 'Reported', 'Clinical_classification',
       'Variant_remarks', 'Reference', 'Variant_Type', 'RefSeq mRNA ID',
       'Chromosome/scaffold name', 'pos', 'ref', 'alt'],
      dtype='object')

In [101]:
lovd_variants['hgvs_g'] = lovd_variants.apply(lambda x: "{}:{}".format(x['Chromosome/scaffold name'], x['DNA_change_genomic_hg19']), axis=1)

In [102]:
lovd_variants.head(4)

,DB_ID,Gene,Transcript,DNA_change_cDNA,DNA_change_genomic_hg19,Reported,Clinical_classification,Variant_remarks,Reference,Variant_Type,RefSeq mRNA ID,Chromosome/scaffold name,pos,ref,alt,hgvs_g
0,KIF21A_000001,KIF21A,NM_017641.3,c.4497T>C,g.39698647A>G,1,benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39698647,A,G,12:g.39698647A>G
1,KIF21A_000002,KIF21A,NM_017641.3,c.3920G>A,g.39709031C>T,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39709031,C,T,12:g.39709031C>T
2,KIF21A_000003,KIF21A,NM_017641.3,c.3909-15G>T,g.39709057C>A,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39709057,C,A,12:g.39709057C>A
3,KIF21A_000004,KIF21A,NM_017641.3,c.1841G>T,g.39735348C>A,3,benign,"VKGL data sharing initiative Nederland, 174 he...","Faruq 2020, submtted",SNP,NM_017641,12,39735348,C,A,12:g.39735348C>A


In [105]:
lovd_variants.query('Variant_Type != "SNP"').hgvs_g.to_csv("/Users/himanshujoshi/Desktop/LOVD variants/lovd_indels.csv", index=False, header=False)

Once the indels have been run through VEP, download the VCF. The bits we're interested in can be obtained as follows:

awk -F "\t" 'NR >= 4 { print $3,$1,$2,$4,$5 }' OFS="\t" lovd_indels.vcf > lovd_indels.vcf.subset.tsv

## Import VCF subset

In [165]:
indels_vcf = pd.read_table("/Users/himanshujoshi/Desktop/LOVD variants/lovd_indels.vcf.subset.tsv", sep="\t", header=0)

In [167]:
indels_vcf.shape

(50075, 5)

In [168]:
indels_vcf.drop_duplicates(inplace=True)

In [169]:
indels_vcf.shape

(44892, 5)

In [170]:
indels_vcf.rename(columns={'ID':'hgvs_g',
                          '#CHROM':'vcf_chrom',
                          'POS': 'vcf_pos',
                          'REF': 'vcf_ref',
                          'ALT': 'vcf_alt'}, inplace=True)

# Combine indels with lovd_variants

In [176]:
lovd_variants2 = lovd_variants.merge(indels_vcf, on='hgvs_g', how='left')

In [177]:
lovd_variants2.head()

,DB_ID,Gene,Transcript,DNA_change_cDNA,DNA_change_genomic_hg19,Reported,Clinical_classification,Variant_remarks,Reference,Variant_Type,RefSeq mRNA ID,Chromosome/scaffold name,pos,ref,alt,hgvs_g,vcf_chrom,vcf_pos,vcf_ref,vcf_alt
0,KIF21A_000001,KIF21A,NM_017641.3,c.4497T>C,g.39698647A>G,1,benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39698647,A,G,12:g.39698647A>G,NaN,NaN,NaN,NaN
1,KIF21A_000002,KIF21A,NM_017641.3,c.3920G>A,g.39709031C>T,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39709031,C,T,12:g.39709031C>T,NaN,NaN,NaN,NaN
2,KIF21A_000003,KIF21A,NM_017641.3,c.3909-15G>T,g.39709057C>A,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39709057,C,A,12:g.39709057C>A,NaN,NaN,NaN,NaN
3,KIF21A_000004,KIF21A,NM_017641.3,c.1841G>T,g.39735348C>A,3,benign,"VKGL data sharing initiative Nederland, 174 he...","Faruq 2020, submtted",SNP,NM_017641,12,39735348,C,A,12:g.39735348C>A,NaN,NaN,NaN,NaN
4,KIF21A_000005,KIF21A,NM_017641.3,c.1674-11C>G,g.39735937G>C,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,NM_017641,12,39735937,G,C,12:g.39735937G>C,NaN,NaN,NaN,NaN


In [178]:
lovd_variants2['vcf_chrom'] = np.where(lovd_variants2['vcf_chrom'].isna(), lovd_variants2['Chromosome/scaffold name'], lovd_variants2['vcf_chrom'])
lovd_variants2['vcf_pos'] = np.where(lovd_variants2['vcf_pos'].isna(), lovd_variants2['pos'], lovd_variants2['vcf_pos'])
lovd_variants2['vcf_ref'] = np.where(lovd_variants2['vcf_ref'].isna(), lovd_variants2['ref'], lovd_variants2['vcf_ref'])
lovd_variants2['vcf_alt'] = np.where(lovd_variants2['vcf_alt'].isna(), lovd_variants2['alt'], lovd_variants2['vcf_alt'])

## Remove unnecessary columns

In [179]:
lovd_variants2.drop(columns=['RefSeq mRNA ID','Chromosome/scaffold name','pos','ref','alt','hgvs_g'], inplace=True)

In [180]:
lovd_variants2.head()

,DB_ID,Gene,Transcript,DNA_change_cDNA,DNA_change_genomic_hg19,Reported,Clinical_classification,Variant_remarks,Reference,Variant_Type,vcf_chrom,vcf_pos,vcf_ref,vcf_alt
0,KIF21A_000001,KIF21A,NM_017641.3,c.4497T>C,g.39698647A>G,1,benign,VKGL data sharing initiative Nederland,-,SNP,12,39698647,A,G
1,KIF21A_000002,KIF21A,NM_017641.3,c.3920G>A,g.39709031C>T,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,12,39709031,C,T
2,KIF21A_000003,KIF21A,NM_017641.3,c.3909-15G>T,g.39709057C>A,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,12,39709057,C,A
3,KIF21A_000004,KIF21A,NM_017641.3,c.1841G>T,g.39735348C>A,3,benign,"VKGL data sharing initiative Nederland, 174 he...","Faruq 2020, submtted",SNP,12,39735348,C,A
4,KIF21A_000005,KIF21A,NM_017641.3,c.1674-11C>G,g.39735937G>C,1,likely benign,VKGL data sharing initiative Nederland,-,SNP,12,39735937,G,C


In [203]:
def valid_bases(bases: str) -> bool:
    if re.match("^[ATCG]+$", bases):
        return True
    else:
        return False

In [210]:
lovd_variants2['Valid_bases'] = lovd_variants2.apply(lambda x: 'Y' if valid_bases(str(x['vcf_ref'])) and valid_bases(str(x['vcf_alt'])) else 'N', axis=1)

# Output data

In [215]:
lovd_variants2.to_csv("/Users/himanshujoshi/Desktop/LOVD variants/all_LOVD_variants_with_VCF.tsv.gz", sep="\t", compression='gzip', index=False)

In [214]:
lovd_variants2.query('Valid_bases == "N"').sort_values(by='vcf_ref')

,DB_ID,Gene,Transcript,DNA_change_cDNA,DNA_change_genomic_hg19,Reported,Clinical_classification,Variant_remarks,Reference,Variant_Type,vcf_chrom,vcf_pos,vcf_ref,vcf_alt,Valid_bases
62757,ABCA4_001010,ABCA4,NM_000350.2,c.3583_3584insNN,g.94505622_94505623insNN,2,VUS,1 more item,"PubMed: Cornelis 2017 , Journal: Cornelis 20...",Other,1,9.45056e+07,A,ANN,N
253284,TSC2_000213,TSC2,NM_000548.3,c.3673_3674insNNNNN,g.2131658_2131659insNNNNN,1,pathogenic,5bp insertion (bases not specified); reported ...,"PubMed: Jones, 1999",Other,16,2.13166e+06,A,ANNNNN,N
97311,AR_000514,AR,NM_000044.3,c.1998A>Y,g.66931356A>Y,2,-,-,"Steinkamp et al. Cancer Res 69: 4434-4442, 2009",Other,X,6.69314e+07,AA,<Y>,N
62421,ABCA4_000663,ABCA4,NM_000350.2,c.3043T>S,g.94510176A>S,3,-,-,PubMed: Passerini 2010,Other,1,9.45102e+07,AA,<S>,N
71236,IL1RAPL1_000017,IL1RAPL1,NM_014271.3,c.1854A>Y,g.29973700A>Y,2,-,-,"PubMed: Piton 2008 , Journal: Piton 2008",Other,X,2.99737e+07,AA,<Y>,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321462,KHDC3L_000005,KCNQ5,NM_019842.3,c.399-4566_*2910112del,g.73709065_76815249del,1,-,decreased gene dosage,"PubMed: DDDS 2015 , Journal: DDDS 2015",Other,6,NaN,NaN,NaN,N
321566,FAM161A_000017,FAM161A,NM_001201543.1,c.1355_1356del,"g.62066781_62066782del, g.62066784_62066785del",4,-,1 more item,"Sharon, submitted",Other,2,NaN,NaN,NaN,N
321611,IMPDH1_000003,PODXL,NM_001018111.2,c.-4332841_*10189006del,g.121000064_135573959del,1,-,decreased gene dosage,"PubMed: DDDS 2015 , Journal: DDDS 2015",Other,7,NaN,NaN,NaN,N
321667,CASK_000037,MAOB,NM_000898.4,c.-326_*1980391del,g.41646322_43741871del,1,-,decreased gene dosage,"PubMed: DDDS 2015 , Journal: DDDS 2015",Other,X,NaN,NaN,NaN,N


In [219]:
pd.crosstab(lovd_variants2.Variant_Type, lovd_variants2.Valid_bases)

Valid_bases,N,Y
Variant_Type,,
Other,11909,49841
SNP,0,260251


In [225]:
lovd_variants2.sample(2).transpose()

,192374,266689
DB_ID,TCAP_000010,SLCO1B3_000010
Gene,TCAP,SLCO1B3
Transcript,NM_003673.3,NM_019844.3
DNA_change_cDNA,c.37_39del,c.550G>A
DNA_change_genomic_hg19,g.37821649_37821651del,g.21015414G>A
Reported,16,1
Clinical_classification,"VUS, benign, likely benign, pathogenic (recess...",VUS
Variant_remarks,"not in 400 control chromosomes, 0/144 DCM pati...",VKGL data sharing initiative Nederland
Reference,"PubMed: Bos 2006 , PubMed: Marziliano 2006 ...",-
Variant_Type,Other,SNP
